In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import get_dummies
from google.colab import files


In [4]:
from google.colab import files
uploaded = files.upload()
uploaded1 = files.upload()

Saving Data_Train.xlsx to Data_Train.xlsx


Saving Data_Test.xlsx to Data_Test.xlsx


In [0]:
import io
dfTest = pd.read_excel(io.BytesIO(uploaded1['Data_Test.xlsx'])) #Loading and viewing the test dataset
dfTrain = pd.read_excel(io.BytesIO(uploaded['Data_Train.xlsx'])) #Loading and viewing the train dataset

In [31]:
dfTest.dropna(inplace = True)
dfTrain.dropna(inplace = True)
dfTest['Price'] = 1e8
print(dfTest.head(), dfTrain.head())

                                        Name    Location  ...  Seats        Price
0                    Maruti Alto K10 LXI CNG       Delhi  ...    4.0  100000000.0
1              Maruti Alto 800 2016-2019 LXI  Coimbatore  ...    5.0  100000000.0
2  Toyota Innova Crysta Touring Sport 2.4 MT      Mumbai  ...    7.0  100000000.0
3                       Toyota Etios Liva GD   Hyderabad  ...    5.0  100000000.0
4                          Hyundai i20 Magna      Mumbai  ...    5.0  100000000.0

[5 rows x 12 columns]                                Name    Location  Year  ...      Power Seats  Price
0            Maruti Wagon R LXI CNG      Mumbai  2010  ...  58.16 bhp   5.0   1.75
1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015  ...  126.2 bhp   5.0  12.50
2                      Honda Jazz V     Chennai  2011  ...   88.7 bhp   5.0   4.50
3                 Maruti Ertiga VDI     Chennai  2012  ...  88.76 bhp   7.0   6.00
4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013  ...  140.8 bhp 

In [77]:
frames = [dfTrain, dfTest]
dfCombined = pd.concat(frames)
dfCombined.reset_index(drop = True, inplace = True)
dfCombined.shape

(7198, 12)

In [78]:
print(dfCombined.head())
print(dfCombined.tail())

                               Name    Location  Year  ...      Power Seats  Price
0            Maruti Wagon R LXI CNG      Mumbai  2010  ...  58.16 bhp   5.0   1.75
1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015  ...  126.2 bhp   5.0  12.50
2                      Honda Jazz V     Chennai  2011  ...   88.7 bhp   5.0   4.50
3                 Maruti Ertiga VDI     Chennai  2012  ...  88.76 bhp   7.0   6.00
4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013  ...  140.8 bhp   5.0  17.74

[5 rows x 12 columns]
                                                   Name  ...        Price
7193                  Volkswagen Vento Diesel Trendline  ...  100000000.0
7194                             Volkswagen Polo GT TSI  ...  100000000.0
7195                             Nissan Micra Diesel XV  ...  100000000.0
7196                             Volkswagen Polo GT TSI  ...  100000000.0
7197  Mercedes-Benz E-Class 2009-2013 E 220 CDI Avan...  ...  100000000.0

[5 rows x 12 columns]


In [0]:
dfCombined['Mileage'] = dfCombined['Mileage'].apply(lambda x: float(list(x.split(' '))[0]))
dfCombined['Engine'] = dfCombined['Engine'].apply(lambda x: float(list(x.split(' '))[0]))  #Cleaning the data a little bit
dfCombined['Power'] = dfCombined['Power'].apply(lambda x: list(x.split(' '))[0]) #from experience, power has a 'null' string entry
lisNull = [index for index, element in enumerate(dfCombined['Power']) if element == 'null']
dfCombined.drop(lisNull, inplace = True)
dfCombined['Power'] = dfCombined['Power'].astype('float64')

In [80]:
dfCombined.head()
dfCombined.shape

(7073, 12)

In [87]:
#Vectorising the text data, lemmatisation and other advanced text normalisation wasn't done because of obvious reasons

textData = dfCombined['Name']

# from sklearn.feature_extraction.text import TfidfVectorizer
# vectoriser = TfidfVectorizer()
# textData = vectoriser.fit_transform(textData)

from sklearn.feature_extraction.text import CountVectorizer
vectoriser = CountVectorizer()
textData = vectoriser.fit_transform(textData).toarray()

from sklearn.decomposition import TruncatedSVD

svDec = TruncatedSVD(n_components=28, n_iter=100)

textDataTransformed = svDec.fit_transform(textData)
textDataTransformed.shape

dfCombined.drop(['Name'], axis = 1, inplace = True)

dfCombined = pd.concat([dfCombined, pd.DataFrame(textDataTransformed)], axis = 1)
dfCombined.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,Mumbai,2010.0,72000.0,CNG,Manual,First,26.60,998.0,58.16,5.0,1.75,0.708541,0.890538,-0.018754,-0.037542,-0.032304,-0.025059,0.015544,-0.026683,-0.001775,0.003649,-0.012526,-0.828587,0.122986,-0.055219,-0.103254,-0.103396,-0.064771,0.498974,0.004301,-0.054139,-0.007069,-0.095247,-0.089855,0.089250,-0.263665,-0.042261,0.053194,-0.407002
1,Pune,2015.0,41000.0,Diesel,Manual,First,19.67,1582.0,126.20,5.0,12.50,1.093638,-0.878000,-0.067181,-0.063036,-0.082576,-0.033883,0.025142,-0.086480,0.042733,-0.018091,1.001436,0.017810,-0.082744,-0.173702,-0.087461,-0.396849,-0.004963,0.025653,0.048615,0.006277,-0.019929,0.389881,-0.185517,-0.293009,0.234229,0.063040,0.093483,0.051658
2,Chennai,2011.0,46000.0,Petrol,Manual,First,18.20,1199.0,88.70,5.0,4.50,0.026274,-0.010671,0.068353,0.741663,-0.367577,0.041758,0.089598,-0.109392,-0.140111,-0.023232,0.029665,-0.003229,0.021679,0.108248,0.041416,0.031115,-0.140545,0.006544,-0.025020,0.017521,-0.000837,-0.028596,0.012177,0.005358,-0.051482,-0.010249,0.135770,0.264931
3,Chennai,2012.0,87000.0,Diesel,Manual,First,20.77,1248.0,88.76,7.0,6.00,0.673084,0.847322,-0.018042,-0.039804,-0.038418,-0.010015,0.027317,-0.035060,-0.019858,-0.004816,-0.006133,0.247314,-0.055685,-0.009880,0.061539,0.045326,0.039169,0.316430,-0.009404,-0.023077,-0.002678,0.477782,-0.108560,0.342924,0.124177,-0.020870,-0.194021,0.190595
4,Coimbatore,2013.0,40670.0,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,17.74,0.042945,-0.012174,0.242124,0.150505,0.563613,1.026138,-0.379735,-0.451421,-0.105800,-0.042393,-0.134606,0.012663,-0.027783,0.000993,-0.314532,-0.281956,-0.183131,-0.011021,-0.032557,-0.066464,-0.002489,0.090506,-0.138737,0.058240,-0.055324,0.102370,-0.093456,0.092464


In [0]:
mEnc = pd.get_dummies(dfCombined, columns = ['Location', 'Fuel_Type', 'Transmission', 'Owner_Type'])

In [89]:
mEnc.head()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,Location_Ahmedabad,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_LPG,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third
0,2010.0,72000.0,26.60,998.0,58.16,5.0,1.75,0.708541,0.890538,-0.018754,-0.037542,-0.032304,-0.025059,0.015544,-0.026683,-0.001775,0.003649,-0.012526,-0.828587,0.122986,-0.055219,-0.103254,-0.103396,-0.064771,0.498974,0.004301,-0.054139,-0.007069,-0.095247,-0.089855,0.089250,-0.263665,-0.042261,0.053194,-0.407002,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0
1,2015.0,41000.0,19.67,1582.0,126.20,5.0,12.50,1.093638,-0.878000,-0.067181,-0.063036,-0.082576,-0.033883,0.025142,-0.086480,0.042733,-0.018091,1.001436,0.017810,-0.082744,-0.173702,-0.087461,-0.396849,-0.004963,0.025653,0.048615,0.006277,-0.019929,0.389881,-0.185517,-0.293009,0.234229,0.063040,0.093483,0.051658,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
2,2011.0,46000.0,18.20,1199.0,88.70,5.0,4.50,0.026274,-0.010671,0.068353,0.741663,-0.367577,0.041758,0.089598,-0.109392,-0.140111,-0.023232,0.029665,-0.003229,0.021679,0.108248,0.041416,0.031115,-0.140545,0.006544,-0.025020,0.017521,-0.000837,-0.028596,0.012177,0.005358,-0.051482,-0.010249,0.135770,0.264931,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0
3,2012.0,87000.0,20.77,1248.0,88.76,7.0,6.00,0.673084,0.847322,-0.018042,-0.039804,-0.038418,-0.010015,0.027317,-0.035060,-0.019858,-0.004816,-0.006133,0.247314,-0.055685,-0.009880,0.061539,0.045326,0.039169,0.316430,-0.009404,-0.023077,-0.002678,0.477782,-0.108560,0.342924,0.124177,-0.020870,-0.194021,0.190595,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0
4,2013.0,40670.0,15.20,1968.0,140.80,5.0,17.74,0.042945,-0.012174,0.242124,0.150505,0.563613,1.026138,-0.379735,-0.451421,-0.105800,-0.042393,-0.134606,0.012663,-0.027783,0.000993,-0.314532,-0.281956,-0.183131,-0.011021,-0.032557,-0.066464,-0.002489,0.090506,-0.138737,0.058240,-0.055324,0.102370,-0.093456,0.092464,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0


In [90]:
mEnc.shape

(7197, 56)

In [0]:
mEnc.dropna(inplace = True)
mEnc.reset_index(drop = True, inplace = True)

In [92]:
from scipy import stats
zScore = np.abs(stats.zscore(mEnc['Kilometers_Driven']))
R = mEnc[zScore < 3]
R.info()
zScore = np.abs(stats.zscore(R['Mileage']))
R = R[zScore < 3]
# R.info()
zScore = np.abs(stats.zscore(R['Engine']))
R = R[zScore < 3]
# R.info()
zScore = np.abs(stats.zscore(R['Power']))
R = R[zScore < 3]
R.fillna(1e8, inplace = True)
RN = R[R['Price']==1e8]
RP = R[R['Price']!= 1e8]
print(RN.shape, RP.shape)
RN.reset_index(drop = True, inplace = True)
RP.reset_index(drop=True, inplace = True)

zScore = np.abs(stats.zscore(RP['Price']))
RP = RP[zScore < 3]

framesNew = [RN, RP]
rEnc = pd.concat(framesNew)
rEnc.reset_index(drop=True, inplace = True)
rEnc.info()

rEncY = rEnc.iloc[:, 6]
rEnc.drop(['Price'], axis = 1, inplace= True)
rEnc.head()
rEnc = rEnc.apply(lambda x: (x-np.mean(x))/(np.std(x)))
rEnc.head()

rEnc['Price'] = rEncY
rEnc.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 6941 entries, 0 to 6948
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       6941 non-null   float64
 1   Kilometers_Driven          6941 non-null   float64
 2   Mileage                    6941 non-null   float64
 3   Engine                     6941 non-null   float64
 4   Power                      6941 non-null   float64
 5   Seats                      6941 non-null   float64
 6   Price                      6941 non-null   float64
 7   0                          6941 non-null   float64
 8   1                          6941 non-null   float64
 9   2                          6941 non-null   float64
 10  3                          6941 non-null   float64
 11  4                          6941 non-null   float64
 12  5                          6941 non-null   float64
 13  6                          6941 non-null   float

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,Location_Ahmedabad,Location_Bangalore,Location_Chennai,Location_Coimbatore,Location_Delhi,Location_Hyderabad,Location_Jaipur,Location_Kochi,Location_Kolkata,Location_Mumbai,Location_Pune,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_LPG,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third,Price
0,-0.136709,-0.093197,1.525683,-1.489205,-1.410520,-0.372011,-0.791522,-0.090483,-0.048343,2.748490,-1.906985,0.155526,0.452992,-0.648301,-0.873169,-0.343992,0.178225,-0.030977,-0.055417,0.643990,0.373968,0.267407,-1.210215,0.051635,-0.300545,0.391939,-0.225117,-0.451644,-0.760831,0.129934,0.794094,0.401233,-0.811737,-1.632307,-0.203464,-0.246508,-0.298719,2.974095,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,-0.390914,-0.344531,-0.082754,-1.051825,-0.042627,1.070171,-0.583866,0.583866,-2.158596,-0.036908,2.314180,-0.135337,100000000.0
1,1.134503,-0.698360,-1.189595,1.600497,0.980964,2.135603,-0.419793,-0.323447,-0.120867,1.987716,-1.489633,0.127249,0.284512,-0.275061,-0.630086,-0.342371,2.007793,0.002259,-0.240766,0.412315,0.603771,-1.020213,-1.298235,0.047503,-0.279093,0.305379,-0.141777,0.065686,-0.266552,-0.786375,-1.198174,-0.229460,1.498621,4.075481,-0.203464,-0.246508,-0.298719,-0.336237,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,2.558105,-0.344531,-0.082754,0.950729,-0.042627,-0.934430,-0.583866,0.583866,0.463264,-0.036908,-0.432119,-0.135337,100000000.0
2,0.181094,-0.846012,-0.001969,-0.713394,-0.564577,-0.372011,-0.856187,-0.076438,0.097476,-0.453855,-0.183913,0.047967,0.345509,0.021816,-0.781270,6.672628,0.158371,0.040988,-0.197119,0.179493,-0.094948,0.120028,0.064007,-0.020057,-0.211624,-0.026313,-0.360707,-0.081118,0.185228,-0.013166,0.077757,-0.159784,0.130835,0.103760,-0.203464,-0.246508,-0.298719,-0.336237,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,2.558105,-0.344531,-0.082754,-1.051825,-0.042627,1.070171,-0.583866,0.583866,0.463264,-0.036908,-0.432119,-0.135337,100000000.0
3,0.816700,0.825667,-0.617958,1.186473,0.795356,2.135603,-0.772076,-0.077654,-0.038671,2.471488,-1.513020,0.032167,0.095736,-0.537720,0.611724,-0.102432,-0.318996,0.006759,-0.296496,-1.159980,-0.581580,-0.578444,2.724568,-0.203332,-0.010533,-0.344591,0.179359,0.690187,1.580371,0.138049,-1.315301,-0.594244,1.182622,1.291575,-0.203464,-0.246508,-0.298719,2.974095,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,-0.390914,-0.344531,-0.082754,0.950729,-0.042627,-0.934430,-0.583866,0.583866,-2.158596,-0.036908,2.314180,-0.135337,100000000.0
4,0.498897,0.039896,-1.468022,2.740030,1.478298,2.135603,-0.772519,-0.098622,4.817432,-1.278082,-0.610822,-0.311930,0.139927,-0.503260,-0.084938,-0.526108,0.056905,-0.018334,-0.173597,-0.121845,0.101390,-0.080817,-0.065226,-0.055172,-0.022702,0.053374,-0.273925,-0.000820,-0.116410,0.017860,-0.081100,0.010903,-0.038768,-0.043108,-0.203464,-0.246508,-0.298719,-0.336237,-0.317252,-0.370352,-0.276853,-0.345079,-0.316674,-0.390914,2.902499,-0.082754,0.950729,-0.042627,-0.934430,1.712721,-1.712721,0.463264,-0.036908,-0.432119,-0.135337,100000000.0


In [0]:
rEnc.to_csv('combibedSetNormalisedZScoreDimensionReduced.csv')

In [65]:
from google.colab import drive
drive.mount('drive', force_remount=True)
!cp combibedSetNormalisedZScoreDimensionReduced.csv "drive/My Drive/"

KeyboardInterrupt: ignored

In [0]:
#Building, Training the model, and Testing on the new processed data

In [0]:
rEncTrain = rEnc[rEnc['Price'] != 1e8]
rEncTest = rEnc[rEnc['Price']==1e8]
# print(np.max(rEncTrain['Price']), np.max(rEncTest))

In [95]:
rEncTrain.shape, rEncTest.shape

((5564, 56), (1052, 56))

In [96]:
rEncTrain.isnull().any(), rEncTest.isnull().any()

(Year                         False
 Kilometers_Driven            False
 Mileage                      False
 Engine                       False
 Power                        False
 Seats                        False
 0                            False
 1                            False
 2                            False
 3                            False
 4                            False
 5                            False
 6                            False
 7                            False
 8                            False
 9                            False
 10                           False
 11                           False
 12                           False
 13                           False
 14                           False
 15                           False
 16                           False
 17                           False
 18                           False
 19                           False
 20                           False
 21                         

In [97]:
rEncTrain.dropna(axis = 1, inplace = True)
rEncTest.dropna(axis = 1, inplace = True)
# del rEncTest['Price']
X = rEncTrain.iloc[:, :-1]
y = rEncTrain.iloc[:, -1]
print(rEncTest.shape, X.shape, y.shape)

(1052, 56) (5564, 55) (5564,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
print(rEncTest.shape, X.shape, y.shape)

(1052, 56) (5564, 55) (5564,)


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import keras
from sklearn.model_selection import train_test_split
from keras.regularizers import l2

In [0]:
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2)

In [101]:
XTrain.shape, XTest.shape, yTrain.shape, yTest.shape

((4451, 55), (1113, 55), (4451,), (1113,))

In [147]:
model1 = Sequential()
model1.add(Dense(128, input_dim ializer = 'normal', activation = 'relu'))
model1.add(Dense(1, kernel_initializer='normal', activation='linear'))
opt = keras.optimizers.Adam(learning_rate = 0.001)
opt1 = keras.optimizers.Adagrad(= XTrain.shape[1], kernel_initializer='normal', activation = 'relu'))
model1.add(Dense(128, kernel_initializer='normal', activation = 'relu'))
model1.add(Dense(64, kernel_initlr = 0.01)
opt2 = keras.optimizers.Adadelta(lr=0.1)

model1.compile(loss = 'mean_squared_error', optimizer = opt, metrics = ['mean_squared_error', 'mean_absolute_error'])
model1.summary()
history = model1.fit(XTrain, yTrain, epochs= 250 , batch_size= 64,  verbose=1, validation_split=0.1)

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_138 (Dense)            (None, 128)               7168      
_________________________________________________________________
dense_139 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_140 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 65        
Total params: 32,001
Trainable params: 32,001
Non-trainable params: 0
_________________________________________________________________
Train on 4005 samples, validate on 446 samples
Epoch 1/250
4005/4005 [==============================] - 0s 64us/step - loss: 54.0868 - mean_squared_error: 54.0868 - mean_absolute_error: 4.7516 - val_loss: 12.7427 - val_mean_squared_error: 12.7427 

In [148]:
yPred = model1.predict(XTrain)
yPredTest = model1.predict(XTest)
r2_score(yTrain, yPred), r2_score(yTest, yPredTest)

(0.9777416472840017, 0.7905354198355601)

In [0]:
# Explore : Regularisation techniques
# Text mining
# Feature selection
# Dimensionality reduction